In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from inverse_modelling_tfo.data.intensity_interpolation import *

PATH = r'/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/s_based_intensity_low_conc3.pkl'
data = pd.read_pickle(PATH)

data = interpolate_exp(data, weights=(1.0, 0.7))
data["Interpolation Error"] = np.abs(data["Intensity"] - data["Interpolated Intensity"])


ImportError: cannot import name 'get_interpolate_fit_params_custom' from 'inverse_modelling_tfo.data.intensity_interpolation' (/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/intensity_interpolation.py)

In [3]:

print(data.groupby("SDD")["Interpolation Error"].describe())

       count          mean           std           min           25%  \
SDD                                                                    
10   30000.0  2.445256e+04  1.484555e+04  5.475663e-01  1.178747e+04   
14   30000.0  2.115546e+03  3.949005e+03  7.314303e+01  3.705574e+02   
19   30000.0  7.298722e+02  2.348999e+03  2.856693e-02  3.085479e-01   
23   30000.0  2.121955e+01  6.660185e+01  1.854692e-06  1.488235e-03   
28   30000.0  3.868347e+01  1.261091e+02  1.823188e-11  4.254605e-05   
32   30000.0  1.249986e+02  4.055088e+02  5.539993e-12  4.502863e-06   
37   30000.0  6.481807e+01  2.066681e+02  7.705300e-15  3.132271e-07   
41   30000.0  3.744733e+01  1.198372e+02  4.035525e-15  7.067115e-07   
46   30000.0  4.130283e+00  1.256938e+01  1.499486e-16  2.679738e-07   
50   30000.0  1.188987e+01  3.908845e+01  1.642189e-17  1.767821e-07   
55   30000.0  1.810374e+01  5.950485e+01  6.941275e-17  1.346177e-06   
59   30000.0  1.442461e+01  4.589946e+01  7.072669e-18  7.412721

In [2]:
data.columns

Index(['Wave Int', 'SDD', 'Uterus Thickness', 'Maternal Wall Thickness',
       'Maternal Hb Concentration', 'Maternal Saturation',
       'Fetal Hb Concentration', 'Fetal Saturation', 'Intensity',
       'Interpolated Intensity', 'Interpolation Error'],
      dtype='object')